In [1]:
import pandas as pd
import numpy as np

[1] 有一整数列A的DataFrame，删除数值重复的行。

In [2]:
df = pd.DataFrame({'A': [1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7]})
df

,A
0,1
1,2
2,2
3,3
4,4
5,5
6,5
7,5
8,6
9,7


In [5]:
df1 = df.drop_duplicates(subset='A')
df1

,A
0,1
1,2
3,3
4,4
5,5
8,6
9,7


[2] 一个全数值DataFrame，每个数字减去该行的平均值

#关于axis的理解#
1、pandas的axis与numpy的axis在理解上都是一样的，axis = 0代表高维在变，低维不变，即按照列操作；axis = 1代表高维不变，低维在变，即按照行操作。
2、对于删除drop、dropna操作，理解稍微难一点，其实原理是一样的，当axis = 0时，看似按行操作，其实是按列删除值；当axis = 1时，看似按列操作，其实是按行删除值。

In [8]:
df = pd.DataFrame(np.random.random(size = (5,3)))
df

,0,1,2
0,0.843133,0.057569,0.900978
1,0.030293,0.936953,0.447294
2,0.876765,0.809084,0.291849
3,0.937832,0.426077,0.086207
4,0.293142,0.656893,0.028370


In [10]:
df1 = df.sub(df.mean(axis=1),axis=0)
df1

,0,1,2
0,0.242573,-0.542991,0.300418
1,-0.441221,0.465440,-0.024219
2,0.217533,0.149851,-0.367384
3,0.454460,-0.057295,-0.397165
4,-0.032993,0.330758,-0.297765


[3] 一个有5列的DataFrame，求哪一列的和最小

df.sum().idxmin()
其中，df.sum() == df.sum(axis=0)

若求哪一行的和最大： df.sum(axis=1).idxmax()

In [14]:
df = pd.DataFrame(np.random.random(size=(5,5)), columns=['A','B','C','D','E'])
df

,A,B,C,D,E
0,0.916013,0.693074,0.272637,0.858432,0.226036
1,0.956046,0.603912,0.667273,0.696792,0.314213
2,0.336775,0.257539,0.841755,0.761252,0.808567
3,0.804221,0.122104,0.000750,0.550868,0.266940
4,0.575983,0.151185,0.675758,0.260711,0.907737


In [23]:
df.sum(axis=1).idxmin()

3

[4] 给定DataFrame， 求A列每个值的前三大的B的和。

df.groupby('A')['B'].nlargest(3).sum(level=0)

In [26]:
df = pd.DataFrame({'A': list('aaabbcaabcccbbc'), 
                   'B': [12,345,3,1,45,14,4,52,54,23,235,21,57,3,87]})
print(df)

df1 = df.groupby('A')['B'].nlargest(3).sum(level=0)
print(df1)

    A    B
0   a   12
1   a  345
2   a    3
3   b    1
4   b   45
5   c   14
6   a    4
7   a   52
8   b   54
9   c   23
10  c  235
11  c   21
12  b   57
13  b    3
14  c   87
A
a    409
b    156
c    345
Name: B, dtype: int64


[5] 给定DataFrame，有列A,B，A的值在1-100（含），对A列每10步长，求对应的B的和


In [31]:
df = pd.DataFrame({'A': [1,2,11,11,33,34,35,40,79,99], 
                   'B': [1,2,11,11,33,34,35,40,79,99]})
print(df)

df1 = df.groupby(pd.cut(df['A'],np.arange(0, 101, 10)))['B'].sum()
print(df1)

    A   B
0   1   1
1   2   2
2  11  11
3  11  11
4  33  33
5  34  34
6  35  35
7  40  40
8  79  79
9  99  99
A
(0, 10]        3
(10, 20]      22
(20, 30]       0
(30, 40]     142
(40, 50]       0
(50, 60]       0
(60, 70]       0
(70, 80]      79
(80, 90]       0
(90, 100]     99
Name: B, dtype: int64


[6] 给定DataFrame，计算每个元素至左边最近的0（或者至开头）的距离，生成新列Y

1. np.r_是按列连接两个矩阵，就是把两矩阵上下相加，要求列数相等，类似于pandas中的concat()。
2. np.c_是按行连接两个矩阵，就是把两矩阵左右相加，要求行数相等，类似于pandas中的merge()。
3. searchsorted（a，v）函数是判断v在a中哪两个a[n-1],a[n]之间，并返回n-1
4. DataFrame.shift(periods=1, freq=None, axis=0)
periods：类型为int，表示移动的幅度，可以是正数，也可以是负数，默认值是1,1就表示移动一次，注意这里移动的都是数据，而索引是不移动的，移动之后没有对应值的，就赋值为NaN。
freq： DateOffset, timedelta, or time rule string，可选参数，默认值为None，只适用于时间序列，如果这个参数存在，那么会按照参数值移动时间索引，而数据值没有发生变化。

In [41]:
df = pd.DataFrame({'X': [7, 2, 0, 3, 4, 2, 5, 0, 3, 4]})
print(df)

# 方法1
izero = np.r_[-1, (df['X'] == 0).to_numpy().nonzero()[0]] #标记0的位置
idx = np.arange(len(df))
df['Y1'] = idx - izero[np.searchsorted(izero - 1, idx) - 1]
print(df)

   X
0  7
1  2
2  0
3  3
4  4
5  2
6  5
7  0
8  3
9  4
   X  Y1
0  7   1
1  2   2
2  0   0
3  3   1
4  4   2
5  2   3
6  5   4
7  0   0
8  3   1
9  4   2


In [48]:
#方法2
x = (df['X'] != 0).cumsum()
print(x)
y = x != x.shift()
df['Y2'] = y.groupby((y != y.shift()).cumsum()).cumsum().apply(int)
print(df)

0    1
1    2
2    2
3    3
4    4
5    5
6    6
7    6
8    7
9    8
Name: X, dtype: int32
   X  Y1  Y2
0  7   1   1
1  2   2   2
2  0   0   0
3  3   1   1
4  4   2   2
5  2   3   3
6  5   4   4
7  0   0   0
8  3   1   1
9  4   2   2


In [55]:
#方法3
df['Y3'] = df.groupby((df['X'] == 0).cumsum()).cumcount()
print(df)
first_zero_idx = (df['X'] == 0).idxmax()
print(first_zero_idx)
df['Y3'].iloc[0:first_zero_idx] += 1
print(df)

   X  Y1  Y2  Y3
0  7   1   1   0
1  2   2   2   1
2  0   0   0   0
3  3   1   1   1
4  4   2   2   2
5  2   3   3   3
6  5   4   4   4
7  0   0   0   0
8  3   1   1   1
9  4   2   2   2
2
   X  Y1  Y2  Y3
0  7   1   1   1
1  2   2   2   2
2  0   0   0   0
3  3   1   1   1
4  4   2   2   2
5  2   3   3   3
6  5   4   4   4
7  0   0   0   0
8  3   1   1   1
9  4   2   2   2


[7] 一个全数值的DataFrame，返回最大S值的坐标

df.unstack().sort_values()[-3:].index.tolist()
df.unstack().nlargest(3).index.tolist()

In [61]:
df = pd.DataFrame(np.random.random(size=(5, 3)))
print(df)
df.unstack().nlargest(3).index.tolist()

          0         1         2
0  0.877091  0.691518  0.331427
1  0.172384  0.717366  0.290083
2  0.842095  0.295000  0.816906
3  0.777544  0.706709  0.450025
4  0.797083  0.799193  0.820108


[(0, 0), (0, 2), (2, 4)]

[8] 给定DataFrame，将负值代替为同组的平均值

In [74]:
df = pd.DataFrame({'grps': list('aaabbcaabcccbbc'), 
                   'vals': [-12,345,3,1,45,14,4,-52,54,23,-235,21,57,3,87]})
print(df)

def replace_negative(group):
    mask = group < 0
    group[mask] = group[~mask].mean()
    return group

df['vals'] = round(df.groupby(['grps'])['vals'].transform(replace_negative),2)
print(df)

   grps  vals
0     a   -12
1     a   345
2     a     3
3     b     1
4     b    45
5     c    14
6     a     4
7     a   -52
8     b    54
9     c    23
10    c  -235
11    c    21
12    b    57
13    b     3
14    c    87
   grps    vals
0     a  117.33
1     a  345.00
2     a    3.00
3     b    1.00
4     b   45.00
5     c   14.00
6     a    4.00
7     a  117.33
8     b   54.00
9     c   23.00
10    c   36.25
11    c   21.00
12    b   57.00
13    b    3.00
14    c   87.00


[9] 创建Series s，将2015所有工作日作为随机值的索引

B business day frequency 工作日频率
C custom business day frequency 自定义工作日频率
D calendar day frequency 日历日频率
W weekly frequency 每周频率
M month end frequency 月末频率
SM semi-month end frequency (15th and end of month) 半月结束频率（15日和月末）
BM business month end frequency 营业月结束频率
CBM custom business month end frequency 自定义营业月结束频率
MS month start frequency 月开始频率
SMS semi-month start frequency (1st and 15th) 半月开始频率（第1天和第15天）
BMS business month start frequency 营业月开始频率
CBMS custom business month start frequency 自定义营业月开始频率
Q quarter end frequency 四分之一结束频率
BQ business quarter end frequency 业务季度结束频率
QS quarter start frequency 季度开始频率
BQS business quarter start frequency 业务季开始频率
A, Y year end frequency 年终频率
BA, BY business year end frequency 业务年度结束频率
AS, YS year start frequency 年开始频率
BAS, BYS business year start frequency 营业年度开始频率
BH business hour frequency 营业时间频率
H hourly frequency 每小时频率
T, min minutely frequency 分钟的频率
S secondly frequency 秒钟的频率
L, ms milliseconds 毫秒
U, us microseconds 微妙
N nanoseconds 纳秒

In [77]:
dti = pd.date_range(start='2015-01-01', end='2015-12-31', freq='B') 
s = pd.Series(np.random.rand(len(dti)), index=dti)

s.head(10)

2015-01-01    0.071413
2015-01-02    0.981466
2015-01-05    0.352396
2015-01-06    0.058753
2015-01-07    0.964013
2015-01-08    0.507681
2015-01-09    0.717002
2015-01-12    0.841247
2015-01-13    0.629900
2015-01-14    0.264473
Freq: B, dtype: float64